# Azure AI Agent service - Connected agents

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import datetime
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import BingGroundingTool, ConnectedAgentTool, FilePurpose, FileSearchTool, ListSortOrder, MessageRole
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

In [2]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 19-Jun-2025 11:05:53


## Settings

In [3]:
load_dotenv()

True

In [4]:
endpoint = os.getenv("PROJECT_ENDPOINT")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

In [5]:
print(dir(project_client))

['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_client', '_config', '_delete_agent', '_deserialize', '_function_tool', '_function_tool_max_retry', '_serialize', '_validate_tools_and_tool_resources', 'close', 'create_agent', 'create_thread_and_process_run', 'create_thread_and_run', 'delete_agent', 'enable_auto_function_calls', 'files', 'get_agent', 'list_agents', 'messages', 'run_steps', 'runs', 'send_request', 'threads', 'update_agent', 'vector_store_file_batches', 'vector_store_files',

## Agents definition

### 1. Stocks price agent

In [6]:
bing = BingGroundingTool(connection_id=os.getenv("AZURE_BING_CONNECTION_ID"))

In [7]:
model="gpt-4.1"

In [8]:
stocksprice_agent = project_client.create_agent(
    model=model,
    name="stocks-price-agent-demo",
    instructions="Retrieve the current stock price for a specified company.",
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"},
    description="Stocks price agent (demo)",
    temperature=0.7,
)

In [9]:
print(f"Created agent, ID: {stocksprice_agent.id}")

Created agent, ID: asst_o6t6xgsVMTky16F5EDJcRRh7


### 2. Currency agent

In [10]:
currency_convert_agent = project_client.create_agent(
    model=model,
    name="currency-convert-agent-demo",
    instructions="Retrieve the current currency exchange rate.",
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"},
    description="Currency convert agent (demo)",
    temperature=0.7,
)

In [11]:
print(f"Created agent, ID: {currency_convert_agent.id}")

Created agent, ID: asst_dw064UWdSrY5W3uinIvVovT7


## 3. RAG agent

In [12]:
DATA_DIR = "data"

pdf_file = os.path.join(DATA_DIR, "2024_Annual_Report.docx")

In [13]:
!ls -lh $pdf_file

-rw-r--r--@ 1 arturoquiroga  staff   1.7M Jun 19 10:34 data/2024_Annual_Report.docx


In [14]:
file = project_client.files.upload_and_poll(file_path=pdf_file,
                                                  purpose=FilePurpose.AGENTS)

print(f"Uploaded file, file ID: {file.id}")

# create a vector store with the file you uploaded
vector_store = project_client.vector_stores.create_and_poll(
    file_ids=[file.id], name="document_vector_store")

print(f"Created vector store, vector store ID: {vector_store.id}")

Uploaded file, file ID: assistant-V2urCi1iitSAQ9gAZRCxLs
Created vector store, vector store ID: vs_1i2HxaUHLIrm6GFqVZMovnnn


In [15]:
# create a file search tool
file_search_tool = FileSearchTool(vector_store_ids=[vector_store.id])

# notices that FileSearchTool as tool and tool_resources must be added or the agent will be unable to search the file
document_pdf_agent = project_client.create_agent(
    model=model,
    name="document-agent-pdf-demo",
    instructions="You are an AI helpful agent to analyse this document which is the Microsoft 2024 annual report",
    tools=file_search_tool.definitions,
    tool_resources=file_search_tool.resources,
    description="PDF document agent (demo)",
    temperature=0.7,
)

print(f"Created agent, agent ID: {document_pdf_agent.id}")

Created agent, agent ID: asst_TQ8YQi3u3TJxHRM36wanIhgd


### 3. Main agent with connected agents

In [16]:
connected_agent_stocks = ConnectedAgentTool(
    id=stocksprice_agent.id,
    name="stockspriceagentconnection",
    description="Get the stock price of a company")

In [17]:
connected_agent_currency = ConnectedAgentTool(
    id=currency_convert_agent.id,
    name="currencyconvertagentconnection",
    description="Convert an amount into another currency")

In [18]:
connected_document = ConnectedAgentTool(
    id=document_pdf_agent.id,
    name="documentagentconnection",
    description="You are an AI helpful agent to analyse this document which is the Microsoft 2024 annual report")

In [19]:
main_agent_demo = project_client.create_agent(
    model=model,
    name="main-agent-demo",
    instructions="You are the main AI agent",
    description="Main agent (demo)",
    tools=[
        connected_agent_stocks.definitions[0],
        connected_agent_currency.definitions[0],
        connected_document.definitions[0],
    ],
)

In [20]:
print(f"Created agent, ID: {main_agent_demo.id}")

Created agent, ID: asst_ZgQHezDxVSbgdf5kip5fKzbI


## List of agents

In [21]:
agents_list = [stocksprice_agent, currency_convert_agent, document_pdf_agent, main_agent_demo]

for agent in agents_list:
    print(
        f"Agent Details:\n"
        f"📛 Name        : {agent.name}\n"
        f"🆔 ID          : {agent.id}\n"
        f"🧠 Model       : {agent.model}\n"
        f"📜 Instructions: {agent.instructions}\n"
    )

Agent Details:
📛 Name        : stocks-price-agent-demo
🆔 ID          : asst_o6t6xgsVMTky16F5EDJcRRh7
🧠 Model       : gpt-4.1
📜 Instructions: Retrieve the current stock price for a specified company.

Agent Details:
📛 Name        : currency-convert-agent-demo
🆔 ID          : asst_dw064UWdSrY5W3uinIvVovT7
🧠 Model       : gpt-4.1
📜 Instructions: Retrieve the current currency exchange rate.

Agent Details:
📛 Name        : document-agent-pdf-demo
🆔 ID          : asst_TQ8YQi3u3TJxHRM36wanIhgd
🧠 Model       : gpt-4.1
📜 Instructions: You are an AI helpful agent to analyse this document which is the Microsoft 2024 annual report

Agent Details:
📛 Name        : main-agent-demo
🆔 ID          : asst_ZgQHezDxVSbgdf5kip5fKzbI
🧠 Model       : gpt-4.1
📜 Instructions: You are the main AI agent



### Quick test

In [22]:
prompt = "Hello."

In [23]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_lkWkBUnLWsun6Vjalro3BvJh
💬 [2] Created message | ID: msg_xBl6GVVB9NJFjGx8Kgy8skk8
🏃 [3] Created run     | ID: run_Ou2VWnW9Gndg6Ratk2mRR77I
⏳ [4] Polling run status...
🔄 Status: RunStatus.COMPLETED

🔍 Step ID details: step_St0YrNdMUX1vv9ae0QZxnLyP, Type: RunStepType.MESSAGE_CREATION
Details step: {'id': 'step_St0YrNdMUX1vv9ae0QZxnLyP', 'object': 'thread.run.step', 'created_at': 1750345571, 'run_id': 'run_Ou2VWnW9Gndg6Ratk2mRR77I', 'assistant_id': 'asst_ZgQHezDxVSbgdf5kip5fKzbI', 'thread_id': 'thread_lkWkBUnLWsun6Vjalro3BvJh', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1750345571, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_bvCgTEeGDXoEDuTpHOnGM6kD'}}, 'usage': {'prompt_tokens': 364, 'completion_tokens': 11, 'total_tokens': 375, 'prompt_token_details': {'cached_tokens': 0}}}

🤖 [5] Agent responses:

💡 → Messag

In [24]:
run

{'id': 'run_Ou2VWnW9Gndg6Ratk2mRR77I', 'object': 'thread.run', 'created_at': 1750345570, 'assistant_id': 'asst_ZgQHezDxVSbgdf5kip5fKzbI', 'thread_id': 'thread_lkWkBUnLWsun6Vjalro3BvJh', 'status': 'completed', 'started_at': 1750345571, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1750345571, 'required_action': None, 'last_error': None, 'model': 'gpt-4.1', 'instructions': 'You are the main AI agent', 'tools': [{'type': 'connected_agent', 'connected_agent': {'name': 'stockspriceagentconnection', 'description': 'Get the stock price of a company', 'id': 'asst_o6t6xgsVMTky16F5EDJcRRh7'}}, {'type': 'connected_agent', 'connected_agent': {'name': 'currencyconvertagentconnection', 'description': 'Convert an amount into another currency', 'id': 'asst_dw064UWdSrY5W3uinIvVovT7'}}, {'type': 'connected_agent', 'connected_agent': {'name': 'documentagentconnection', 'description': 'You are an AI helpful agent to analyse this document which is the Microsoft 2024 annual re

In [25]:
run.usage

{'prompt_tokens': 364, 'completion_tokens': 11, 'total_tokens': 375, 'prompt_token_details': {'cached_tokens': 0}}

### Example 1

In [26]:
prompt = "What is microsoft's share price?"

In [27]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_DdUwdkKhrjs6aVgjdNjNkgZS
💬 [2] Created message | ID: msg_YX1aUzl1h9buELyMqeRo2ViF
🏃 [3] Created run     | ID: run_ksh1Lyf2I3xE9XYMcbi7d2dH
⏳ [4] Polling run status...
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.COMPLETED

🔍 Step ID details: step_dkfhNM6FKCyE2WSgLeBNCDAN, Type: RunStepType.MESSAGE_CREATION
Details step: {'id': 'step_dkfhNM6FKCyE2WSgLeBNCDAN', 'object': 'thread.run.step', 'created_at': 1750345582, 'run_id': 'run_ksh1Lyf2I3xE9XYMcbi7d2dH', 'assistant_id': 'asst_ZgQHezDxVSbgdf5kip5fKzbI', 'thread_id': 'thread_DdUwdkKhrjs6aVgjdNjNkgZS', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1750345583, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_3nei5IiIcbLiuaso9s2mCdPY'}}, 

### Example 2

In [28]:
prompt = "Convert this into € and in Japanese Yen"

In [29]:
# Step 1: Use a thread
#thread = project_client.threads.create()
print(f"🧵 [1] Using existing thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Using existing thread  | ID: thread_DdUwdkKhrjs6aVgjdNjNkgZS
💬 [2] Created message | ID: msg_9PN5KJYH1FBNLIqJp3ftbGG7
🏃 [3] Created run     | ID: run_0LT0QywVC9ldP48LLvDhB4JJ
⏳ [4] Polling run status...
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.COMPLETED

🔍 Step ID details: step_A43ulpeUgQBUHuvXJHYriOKj, Type: RunStepType.MESSAGE_CREATION
Details step: {'id': 'step_A43ulpeUgQBUHuvXJHYriOKj', 'object': 'thread.run.step', 'created_at': 1750345594, 'run_id': 'run_0LT0QywVC9ldP48LLvDhB4JJ', 'assistant_id': 'asst_ZgQHezDxVSbgdf5kip5fKzbI', 'thread_id': 'thread_DdUwdkKhrjs6aVgjdNjNkgZS', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1750345594, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_vPj1K1l5Gms3HSZiXkh1Wj

### Example 3

In [30]:
prompt = "I have 10 Microsoft shares + 5 IBM shares and 7 SFDC shares. What is the total value of my portfolio in cad?"

In [31]:
# Step 1: Create a new thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_1B8rPD9IcD4kgVJYEyD7I7WF
💬 [2] Created message | ID: msg_M5vZS37ak5f2xlAItz7bippv
🏃 [3] Created run     | ID: run_T6vDlwPB68vO7Gu8DoiqFU19
⏳ [4] Polling run status...
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.COMPLETED

🔍 Step ID details: step_peQK7LZYJpQjex2N6PWhC8ps, Type: RunStepType.MESSAGE_CREATION
Details step: {'id': 'step_peQK7LZYJpQjex2N6PWhC8ps', 'object': 'thread.run.step', 'created_at': 175034562

### Example 4

In [32]:
prompt = "What are the highlights for Microsoft from fiscal year 2024 compared with fiscal year 2023?"

In [33]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_uKpY9LejtTmpgUScKKfVqY6W
💬 [2] Created message | ID: msg_icQBGGbVxIA0GIDRL99Z22aY
🏃 [3] Created run     | ID: run_YTZR2cssCbPNHAPVQJf742p8
⏳ [4] Polling run status...
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.COMPLETED

🔍 Step ID details: step_vzrAX7s5alRmxVcfuvTeZciX, Type: RunStepType.MESSAGE_CREATION
Details step: {'id': 'step_vzrAX7s5alRmxVcfuvTeZciX', 'object': 'thread.run.step', 'created_at': 1750345633, 'run_id': 'run_YTZR2cssCbPNHAPVQJf742p8', 'assistant_id': 'asst_ZgQHezDxVSbgdf5kip5fKzbI', 'thread_id': 'thread_uKpY9LejtTmpgUScKKfVqY6W', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1750345634, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'm

### Example 5

In [34]:
prompt = "What are the main products and companies mentionned in the 2024 Microsoft annual report?"

In [35]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_qAar0oMfR7zvXYAAKZ0OPk98
💬 [2] Created message | ID: msg_ZQvEvtUTK3PwTAqnpqgEcuLN
🏃 [3] Created run     | ID: run_Lpng4pcnz6aZO49xW9Kidayz
⏳ [4] Polling run status...
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.IN_PROGRESS
🔄 Status: RunStatus.COMPLETED

🔍 Step ID details: step_kESmR9rkoNwjr6VPY3krQeBU, Type: RunStepType.MESSAGE_CREATION
Details step: {'id': 'step_kESmR9rkoNwjr6VPY3krQeBU', 'object': 'thread.run.step', 'created_at': 1750345646, 'run_id': 'run_Lpng4pcnz6aZO49xW9Kidayz', 'assistant_id': 'asst_ZgQHezDxVSbgdf5kip5fKzbI', 'thread_id': 'thread_qAar0oMfR7zvXYAAKZ0OPk98', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1750345647, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'm

## Post processing

In [36]:
agents_list = [stocksprice_agent, currency_convert_agent, document_pdf_agent, main_agent_demo]

for agent in agents_list:
    print(
        f"Agent Details:\n"
        f"📛 Name        : {agent.name}\n"
        f"🆔 ID          : {agent.id}\n"
        f"🧠 Model       : {agent.model}\n"
        f"📜 Instructions: {agent.instructions}\n"
    )

Agent Details:
📛 Name        : stocks-price-agent-demo
🆔 ID          : asst_o6t6xgsVMTky16F5EDJcRRh7
🧠 Model       : gpt-4.1
📜 Instructions: Retrieve the current stock price for a specified company.

Agent Details:
📛 Name        : currency-convert-agent-demo
🆔 ID          : asst_dw064UWdSrY5W3uinIvVovT7
🧠 Model       : gpt-4.1
📜 Instructions: Retrieve the current currency exchange rate.

Agent Details:
📛 Name        : document-agent-pdf-demo
🆔 ID          : asst_TQ8YQi3u3TJxHRM36wanIhgd
🧠 Model       : gpt-4.1
📜 Instructions: You are an AI helpful agent to analyse this document which is the Microsoft 2024 annual report

Agent Details:
📛 Name        : main-agent-demo
🆔 ID          : asst_ZgQHezDxVSbgdf5kip5fKzbI
🧠 Model       : gpt-4.1
📜 Instructions: You are the main AI agent



In [37]:
for agent in agents_list:
    project_client.delete_agent(agent.id)
    print(f"Deleted agent, agent ID: {agent.id}")

Deleted agent, agent ID: asst_o6t6xgsVMTky16F5EDJcRRh7
Deleted agent, agent ID: asst_dw064UWdSrY5W3uinIvVovT7
Deleted agent, agent ID: asst_TQ8YQi3u3TJxHRM36wanIhgd
Deleted agent, agent ID: asst_ZgQHezDxVSbgdf5kip5fKzbI
